In [1]:
import os
from SimpleITK.SimpleITK import And
import numpy as np
import scipy
import SimpleITK as sitk
import matplotlib.pyplot as plt
from skimage.segmentation import join_segmentations
import pandas as pd


In [7]:
def GetSlices(seg_arr):
    values= []
    for i in range(0,len(seg_arr)):
        val = np.sum(seg_arr[i,:,:])
        if val >=  20000 :
            values.append(i)
    return values

def GetSlicesArray(masks):
    slices_array =[]
    for image in masks:
        slices = GetSlices(image)
        slices_array.append((slices))
    return np.asarray(slices_array)



def StapleGun(masks_a, masks_b, masks_c, ids_a, ids_b, ids_c):

    slices_array_a = GetSlicesArray(masks_a)
    slices_array_b = GetSlicesArray(masks_b)
    slices_array_c = GetSlicesArray(masks_c)
    """
    #removed CR slices, for when I have all of the data sheets 
    path_a = "path"
    path_b = "path"
    path_c = "path"
    slices_array_a = RemoveCoronaRadiataSlices(masks_a, ids_a, path_a)
    slices_array_b = RemoveCoronaRadiataSlices(masks_b, ids_b, path_b)
    slices_array_c = RemoveCoronaRadiataSlices(masks_c, ids_c, path_c) 368.32
    """

    staple_array = []
    slices_array = []
    id = []
    id_ticker = 0
    

    for i in range (0, 77) :
        id_ticker += 1
        staple_slices_array = []  
        slices_id =[]
        patient_id = ids_a[i]

        for j in slices_array_a[i]:
            if j in slices_array_b[i] and j in slices_array_c[i]:

                masks_a[i][j][masks_a[i][j]==255] = 1
                masks_b[i][j][masks_b[i][j]==255] = 1
                masks_c[i][j][masks_c[i][j]==255] = 1

                mask_a = sitk.GetImageFromArray(masks_a[i][j])
                mask_b = sitk.GetImageFromArray(masks_b[i][j])
                mask_c = sitk.GetImageFromArray(masks_c[i][j])
                

                staple_filter = sitk.STAPLEImageFilter()
                staple_filter.SetForegroundValue(1)
                staple_image = staple_filter.Execute(mask_a,mask_b, mask_c)
                staple_image = staple_image > 0.5                                
                staple_slice_array = sitk.GetArrayFromImage(staple_image)
                iter = staple_filter.GetElapsedIterations()
                print("Patient: ", i, ", Slice: ", j)
                print("iter: ", iter)

                staple_slices_array.append(staple_slice_array)
                slices_id.append(j)
                
        id.append((patient_id))
        staple_array.append(staple_slices_array)
        slices_array.append(slices_id)
        
    return staple_array, id, slices_array

def UnpackNpz(path):
    data = np.load(path, allow_pickle=True)
    masks = data['masks']
    ids = data['ids']
    return masks, ids

def OnesNZeros(mask_array):
    mask_array_copy = mask_array
    slices_array = GetSlicesArray(mask_array_copy)
    for i in range(0, len(mask_array_copy)):
        for slic in slices_array[i]:
            mask_array_copy[i][slic][mask_array_copy[i][slic]==255] = 1
    return mask_array_copy

def HasCoronaRadiata(csv_path):
    data_frame = pd.read_excel(csv_path, sheet_name='Sheet1', usecols = ['PRIME_ID ', 'CR_distance_to_hematoma_(mm)'])
    
    CR =data_frame['CR_distance_to_hematoma_(mm)'].tolist()
    ids = data_frame['PRIME_ID '].tolist()
    ind = []
    for i in range(0,len(CR)):
        if CR[i] !=0:
            ind.append(i)
    patients=[]
    for i in ind:
        patient = ids[i][0:4]
        patients.append(patient)

    return np.asarray(patients, dtype = int)

def RemoveCoronaRadiataSlices(mask_array, mask_ids, csv_path):
    mask_ids_dummy = mask_ids.astype(int).tolist()
    patients = HasCoronaRadiata(csv_path)
    slices_array = GetSlicesArray(mask_array)
    for id in patients:
        if id in mask_ids_dummy:
            
            ind = mask_ids_dummy.index(id)
            length= len(slices_array[ind])
            if id != 0:
             slices_array[ind].pop(length-1)
         
    return slices_array

  


def IdSkip(ids_a,ids_b,ids_c, masks_a, masks_b, masks_c):
    mask_a = masks_a.tolist()
    mask_b = masks_b.tolist()
    mask_c = masks_c.tolist()
    id_a = ids_a.tolist()
    id_b = ids_b.tolist()
    id_c = ids_c.tolist()
    ids_skip =[]
    for id in ids_a:
        if id not in ids_b or id not in ids_c:
            ids_skip.append(id)
    for id in ids_skip:
        if id in ids_a:
            i = id_a.index(id)
            mask_a.pop(i)
            id_a.pop(i)

        if id in ids_b:
            i = id_b.index(id)
            mask_b.pop(i)
            id_b.pop(i)

        if id in ids_c:
            i = id_c.index(id)
            mask_c.pop(i)
            id_c.pop(i)

    return np.asarray(mask_a), np.asarray(id_a), np.asarray(mask_b), np.asarray(id_b), np.asarray(mask_c), np.asarray(id_c)

def dice_loss(input, target):
    
    smooth = 1.

    iflat = input.view(-1)
    tflat = target.view(-1)
    print("iflat: ", iflat, "tflat: ", tflat)
    intersection = (iflat * tflat).sum(-1)
    print(intersection)
    print(iflat.sum(-1) + tflat.sum(-1))
    return ((2. * intersection + smooth) /
              (iflat.sum(-1) + tflat.sum(-1) + smooth))


def compute_dice_coefficient(mask_pred, mask_gt):

  volume_sum = mask_gt.sum() + mask_pred.sum()
  if volume_sum == 0:
    return np.NaN
  volume_intersect = (mask_gt & mask_pred).sum()
  return 2*volume_intersect / volume_sum 

def DiceCaster(staple_array, staple_ids, staple_slices, masks_a, ids_a):
    
    id_a = ids_a.tolist()
    dice_array =[]
    for i in range(0, len(staple_array)):
        dices_array =[]
        for j in range(0, len(staple_array[i])):
            patient_id = staple_ids[i]
            slices_arr = staple_slices[i]
            place_holder = id_a.index(patient_id)
            ind = slices_arr[j]
            
            print("place holder: ", place_holder, "ind: ", ind, "i: ", i, "j: ", j )
            dice_coeff = compute_dice_coefficient(masks_a[place_holder][ind,:,:], staple_array[i][j])
            dices_array.append(dice_coeff)
        dice_array.append(dices_array)   
    return dice_array


    
    



In [3]:
adrian_path = '/Users/olivia/Documents/PhD/MISTIE/mask_data/adrian_masks.npz'
sacha_path = '/Users/olivia/Documents/PhD/MISTIE/mask_data/sacha_masks.npz'
paul_path = '/Users/olivia/Documents/PhD/MISTIE/mask_data/paul_masks.npz'

masks_paul, ids_paul = UnpackNpz(paul_path)
masks_adrian, ids_adrian = UnpackNpz(adrian_path)
masks_sacha, ids_sacha = UnpackNpz(sacha_path)


In [5]:
slices_a = GetSlicesArray(masks_adrian)
slices_p =GetSlicesArray(masks_paul)
slices_s = GetSlicesArray(masks_sacha)


/var/folders/j9/4t_m1wxx3lnbsv72kfd1kgzc0000gr/T/ipykernel_39528/3763468973.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(slices_array)


In [8]:
'''
slices_a = GetSlicesArray(masks_adrian)
slices_p =GetSlicesArray(masks_paul)
slices_s = GetSlicesArray(masks_sacha)
print(slices_a[0], slices_p[0], slices_s[0])
masks_adrian[1][17]
'''
masks_adrianexp = masks_adrian
masks_paulexp = masks_paul
masks_sachaexp= masks_sacha
#masks_adrianexp[1][17][masks_adrianexp[1][17]==255] = 1
#masks_paulexp[1][17][masks_paulexp[1][17]==255] = 1
#masks_paulexp[1][17][masks_paulexp[1][17]==255] = 1
"""
staple_filter = sitk.STAPLEImageFilter()
staple_filter.SetForegroundValue(1)
print('unique: ', np.unique(masks_adrianexp[1][17]))
a = sitk.GetImageFromArray((masks_adrianexp[1][17]))
b = sitk.GetImageFromArray((masks_paulexp[1][17]))
c = sitk.GetImageFromArray((masks_sachaexp[1][17]))
staple_image = staple_filter.Execute(a,b,c)#sitk.GetImageFromArray(masks_adrian[0][26]),sitk.GetImageFromArray(masks_paul[0][26]), sitk.GetImageFromArray(masks_sacha[0][26]))
staple_image = staple_image > 0.5                                
staple_slice_array = sitk.GetArrayFromImage(staple_image)
iter = staple_filter.GetElapsedIterations()
print("staple shape: ", staple_slice_array.shape, np.unique(staple_slice_array))
print("iter: ", iter)
"""
#slicesx = GetSlicesArray(masks_paul)
#print(slicesx[19])
#slices = GetSlices(masks_paul[9])
#print(slices, ids_paul[8])
#plt.imshow(masks_paul[19][37])
#print(ids_paul[19])
path = '/Users/olivia/Documents/PhD/MISTIE/mask_data/pauldatasheet.xlsx'
patients = HasCoronaRadiata(path)
#print(patients)
#print(np.asarray(ids_paul, dtype= int).shape)
slices_noCR = RemoveCoronaRadiataSlices(masks_paul, ids_paul, path)
print(slices_noCR[47])
print(slices_p[47])


2011 47
2025 8
2027 45
2033 77
2049 1
2053 38
2054 37
2058 73
2060 75
2066 81
2075 41
2085 0
2086 42
2107 55
2110 23
2121 21
2134 17
2136 53
2141 66
2142 27
2144 24
2146 67
2151 14
2153 57
2159 19
2165 59
2174 26
2175 25
2178 62
2200 86
2209 83
2212 44
2213 39
2217 9
2223 48
2252 74
2253 80
2257 34
2263 71
2266 36
2272 3
2285 43
2306 18
2316 69
2318 68
2321 63
2338 20
2345 65
2350 58
2351 51
2357 60
2381 61
2389 64
2394 52
2397 16
2401 33
2403 72
2410 50
2413 10
2414 11
2424 2
2431 30
2432 70
2434 82
2445 40
2453 28
2461 78
2463 31
2465 29
2479 7
2484 6
2492 79
2494 84
[9, 10, 11, 12, 13, 14]
[9, 10, 11, 12, 13, 14, 16]


/var/folders/j9/4t_m1wxx3lnbsv72kfd1kgzc0000gr/T/ipykernel_39528/2658534850.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray(slices_array)


In [11]:
#plt.imshow(paul_noCR[47][16])
#plt.imshow(masks_paul[84][16])

print(GetSlices(masks_paul[84]))

[11, 13, 14, 15]


In [ ]:


new_mask_paul,new_id_paul,new_mask_adrian,new_id_adrian,new_mask_sacha,new_id_sacha = IdSkip(ids_paul, ids_adrian, ids_sacha, masks_paul, masks_adrian, masks_sacha)
#print(len(ma), len(ia), len(mb), len(ib), len(mc), len(ic))
#new_mask_adrian = OnesNZeros(new_mask_adrian)
#new_mask_paul = OnesNZeros(new_mask_paul)
#new_mask_sacha = OnesNZeros(new_mask_sacha)
#staple_array, staple_ids, staple_slices = StapleGun(masks_a= new_mask_adrian, masks_b= new_mask_paul, masks_c= new_mask_sacha, ids_a= new_id_adrian, ids_b = new_id_paul, ids_c= new_id_sacha)

In [ ]:
new_mask_adrian_copy = new_mask_adrian
new_mask_paul_copy = new_mask_paul
new_mask_sacha_copy = new_mask_sacha
new_mask_adrian10 = OnesNZeros(new_mask_adrian_copy)
new_mask_paul10 = OnesNZeros(new_mask_paul_copy)
new_mask_sacha10 = OnesNZeros(new_mask_sacha_copy)

In [ ]:
staple_array, staple_ids, staple_slices = StapleGun(masks_a= new_mask_adrian, masks_b= new_mask_paul, masks_c= new_mask_sacha, ids_a= new_id_adrian, ids_b = new_id_paul, ids_c= new_id_sacha)

In [ ]:
masks_adrian_copy = masks_adrian

masks_adrian_01 = OnesNZeros(masks_adrian_copy)
print(np.unique(masks_adrian_01[1][17]))
print((masks_adrian_01[1]).shape)

In [ ]:
print(len(staple_ids)) 
print()
for i in range (0,len(staple_array)):

    for j in range(0, len(staple_array[i])):
            p
            string = (str(i)+  "slice" + str(j))
            print(i,j, staple_array[i][j].shape)
            #plt.imshow(staple_array[i][j])
            #plt.title(string)
            #plt.show()
            
            

In [ ]:
dice_array = DiceCaster(staple_array, staple_ids, staple_slices, new_mask_adrian, new_id_adrian)
print(dice_array[1])
staple_unique_array = staple_array
for i in range(0,len(staple_array)):
    for j in range(0,len(staple_array[i])):

        staple_unique_array[i][j] = np.unique(staple_array[i][j])
"""
print(new_mask_adrian[3][15,:,:].shape)
print(staple_array[4][2].shape)
dice = compute_dice_coefficient(staple_array[4][2], staple_array[4][2])
print(dice)
#print(len(dice_array[4]))

di = compute_dice_coefficient(new_mask_sacha[1][17,:,:], staple_array[1][1])
print(new_mask_sacha[1][17,:,:], staple_array[1][1])
print("unique: ",np.unique((staple_array[1][1] -1)*(-1)))
print(di)
print(slices_s[1])
#plt.imshow(new_mask_sacha[1][17,:,:])
plt.imshow(staple_array[1][1])
plt.imshow(new_mask_sacha[1][17,:,:], alpha = 0.3)
"""

In [ ]:

fig=plt.figure(figsize=(12, 90))
ax = []
rows = 40
columns = 4
#figure with 5 columns

for j in range(0, 75):
  for i in range(0, len(staple_array[j])):
    ax.append(fig.add_subplot(rows, columns, i+1))
    #just cause they look cool 

    plt.imshow(staple_array[j][i], cmap = plt.cm.autumn, alpha = 0.5)
    #plt.imshow(test_predictions[i,0,...], alpha = 0.5)
    plt.axis('off')
  
    plt.show()
    
  
